In [5]:
# creating the classes

class HauntedMansion:
    def __init__(self):
        self.rooms = {}
        self.doors = []

    def add_room(self, room):
        self.rooms[room.name] = room

    def add_door(self, room1_name, room2_name, key_required=None):
        if room1_name in self.rooms and room2_name in self.rooms:
            room1 = self.rooms[room1_name]
            room2 = self.rooms[room2_name]
            door = Door(room1, room2, key_required)
            room1.add_door(door)
            room2.add_door(door)
            self.doors.append(door)
        else:
            print("One or both rooms not found in the house!")

class Room:
    def __init__(self, name, description):
        self.name = name
        self.description = description
        self.objects = {}
        self.doors = []

    def add_door(self, door):
        if door not in self.doors:
            self.doors.append(door)

    def add_object(self, obj):
        self.objects[obj.name] = obj

    def search(self):
        if self.objects:
            print("You found:")
            for obj in self.objects.values():
                print(f"- {obj.name}: {obj.description}")
        else:
            print("There's nothing interesting here.")

class Object:
    def __init__(self, name, description, item=None):
        self.name = name
        self.description = description
        self.item = item

    def interact(self, player):
        if self.item:
            print(f"You found a {self.item} inside the {self.name}!")
            player.inventory.append(self.item)
            self.item = None  # The item is taken
        else:
            print(f"The {self.name} is empty.")

class Item:
    def __init__(self, name, description):
        self.name = name
        self.description = description

class Door:
    def __init__(self, room1, room2, key_required):
        self.room1 = room1
        self.room2 = room2
        self.key_required = key_required

    def get_other_room(self, current_room):
        return self.room2 if current_room == self.room1 else self.room1

class Player:
    def __init__(self):
        self.current_room = None
        self.inventory = []

    def move(self, new_room):
        for door in self.current_room.doors:
            if new_room == door.get_other_room(self.current_room):
                if door.key_required and door.key_required not in self.inventory:
                    print(f"You need a {door.key_required} to go there.")
                    return
                self.current_room = new_room
                print(f"You moved to {new_room.name}.")
                return
        print("There's no door leading there.")
    
    def search_room(self):
        self.current_room.search()

    def interact_with(self, object_name):
        if object_name in self.current_room.objects:
            obj = self.current_room.objects[object_name]
            obj.interact(self)
        else:
            print(f"There's no {object_name} here.")

In [24]:
# creating & setting the game

# Create a house
house = HauntedMansion()

# Create rooms
kitchen = Room("Kitchen", "A warm kitchen with a smell of fresh bread.")
living_room = Room("Living Room", "A cozy living room with a fireplace.")
bedroom = Room("Bedroom", "A peaceful bedroom with a soft bed.")
dining_room = Room("Dining Room", "A nice room to have a meal.")

# Add rooms to the house
house.add_room(kitchen)
house.add_room(living_room)
house.add_room(bedroom)
house.add_room(dining_room)

# Connect rooms with doors
house.add_door("Kitchen", "Living Room", key_required="silver key")
house.add_door("Living Room", "Bedroom")  # No key required for this door
house.add_door("Kitchen", "Dining Room", key_required="gold key")

# Create a player and set their starting room
player = Player()
player.current_room = kitchen

# Create a drawer that contains a key
chest = Object("Chest", "A mysterious chest.", item="gold key")
drawer = Object("Drawer", "An old wooden drawer.", item="silver key")

# Add the drawer to the kitchen
kitchen.add_object(drawer)
kitchen.add_object(chest)

In [25]:
print(f"You are in the {player.current_room.name}.")

You are in the Kitchen.


In [26]:
player.move(living_room)

You need a silver key to go there.


In [27]:
player.search_room()

You found:
- Drawer: An old wooden drawer.
- Chest: A mysterious chest.


In [28]:
player.interact_with("Drawer")

You found a silver key inside the Drawer!


In [29]:
player.move(living_room)

You moved to Living Room.


In [30]:
player.move(bedroom)

You moved to Bedroom.


In [32]:
player.search_room()

There's nothing interesting here.


In [33]:
player.move(dining_room)

There's no door leading there.


In [34]:
player.move(living_room)

You moved to Living Room.


In [35]:
player.move(kitchen)

You moved to Kitchen.


In [36]:
player.move(dining_room)

You need a gold key to go there.


In [37]:
player.search_room()

You found:
- Drawer: An old wooden drawer.
- Chest: A mysterious chest.


In [38]:
player.interact_with("Chest")

You found a gold key inside the Chest!


In [39]:
player.move(dining_room)

You moved to Dining Room.


In [40]:
player.inventory

['silver key', 'gold key']